In [1]:
#!pip show tensorflow_datasets
!pip3 install emnist
!pip3 install tensorflow_datasets
!pip3 install opencv_python
!pip3 install matplotlib
!pip3 install chardet
!pip3 install pandas

In [2]:
# import emnist
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import sklearn
import random
from sklearn.decomposition import PCA 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
import cv2
from tensorflow.keras.layers import Input, Lambda, Conv2D,Conv2DTranspose, MaxPooling2D, BatchNormalization, Dense, Flatten, Activation, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import backend as K
%matplotlib inline
# tf.debugging.set_log_device_placement(False)

In [3]:
print(tf.config.list_physical_devices('GPU'))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
print(tf.test.is_built_with_cuda())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Default GPU Device: /device:GPU:0
True


In [4]:
(ds_train, ds_test), info = tfds.load('omniglot', split=['train', 'test'], with_info=True)

In [5]:
df_train = tfds.as_dataframe(ds_train, info)
df_test  = tfds.as_dataframe(ds_test, info)

In [6]:
train_images = np.stack(df_train['image'])
train_images.shape

(19280, 105, 105, 3)

In [7]:
df_train.columns

Index(['alphabet', 'alphabet_char_id', 'image', 'label'], dtype='object')

In [8]:
len(df_train['label'].unique())

964

In [9]:
# Hyperparameters

shots = 1
train_shots = 5
num_classes = 20
img_size = 56
batch_size = 32
num_mini_iterations = 1000

In [10]:
''' Data handling general functions '''

def separate_fewshot(test_images, test_labels, n=1):
    oneshot_data = []
    classify_data = []
    for label in np.unique(test_labels):
        for num in np.random.choice(np.where(test_labels == label)[0], n, False):
            oneshot_data.append(num)
    temp = set(oneshot_data)
    for i in range(len(test_labels)):
        if not i in temp: classify_data.append(i)
    oneshot_images = test_images[oneshot_data]
    oneshot_labels = test_labels[oneshot_data]
    classify_images = test_images[classify_data]
    classify_labels = test_labels[classify_data]
    return oneshot_images, oneshot_labels, classify_images, classify_labels

In [11]:
def resize_images(images, size, to_grayscale = True):
    resized_images = []
    for img in images:
        resized_image = cv2.resize(img, (size, size))
        if to_grayscale:
            resized_image= cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        resized_images.append(resized_image)
    return np.array(resized_images)

In [12]:
def parse_omniglot_dataframe(df, img_size = 56, reshape=True):
    images = resize_images(df['image'], img_size)
    if reshape:
        images = images.reshape(-1, img_size * img_size)
        images = images.astype("float") / 255.0
    else:
        image = images.reshape(-1, img_size, img_size)
        images = images.astype("float") / 255.0
        images = images[..., np.newaxis]
    labels = df['label'].to_numpy()
    data = {}
    for label in np.unique(labels):
        data[label] = images[np.where(labels == label)]
    return (data, labels)

In [13]:
def get_batch(data, labels, num_iterations=10, num_classes=num_classes, shots=shots, num_support=train_shots, img_size=img_size):
    unique_labels = np.unique(labels)
    num_labels = unique_labels.shape[0]
    np.random.shuffle(unique_labels)
    
    x = np.zeros(shape=(num_classes * shots, img_size, img_size, 1))
    y = np.zeros(shape=(num_classes * num_support, img_size, img_size,1))
    x_l, y_l = np.zeros(num_classes * shots), np.zeros(num_classes * num_support)
    
    size = num_classes * num_classes * shots * num_support
    num_iterations = min(num_iterations, (num_labels + num_classes - 1) // num_classes)
    
    pairs = [[None for _ in range(size * num_iterations)], [None for _ in range(size * num_iterations)]]
    #labels = [[None for _ in range(size * num_iterations)], [None for _ in range(size * num_iterations)]]
    target = np.zeros(size * num_iterations)
    
    cur = 0
    x_label = [None for _ in range(num_iterations * num_classes)]
    for t in range(0, num_iterations * num_classes, num_classes):
        classes = unique_labels[t: t + num_classes]
        #classes = np.random.choice(labels, n_classes, False)        
        for i, c in enumerate(classes):
            x_label[t + i] = c
            ind = np.random.choice(np.arange(data[c].shape[0]), shots + num_support, False)
            x_ind = ind[:shots]
            #print(x_ind.shape, data[c][x_ind].shape, x[i * shots: (i+1) * shots])
            x[i * shots: (i+1) * shots, :][0] = data[c][x_ind]
            x_l[i * shots: (i+1) * shots] = c

            y_ind = ind[shots:]
            y[i * num_support: (i+1) * num_support, :] = data[c][y_ind]
            y_l[i * num_support: (i+1) * num_support] = c
            
        for i, xx in enumerate(x):
            if x_l[i] not in classes:
                continue
            for j, yy in enumerate(y):
                if y_l[j] not in classes:
                    continue
                pairs[0][cur] = xx
                pairs[1][cur] = yy
                target[cur] = ((x_l[i] == y_l[j]) and (x_l[i] != -1))
                cur += 1
    pairs[0] = np.array(pairs[0][:cur])
    pairs[1] = np.array(pairs[1][:cur])
    target = target[:cur]
    return pairs, target, np.array(x_label)

In [14]:
data, lbl = parse_omniglot_dataframe(df_test, reshape=False)
data[lbl[0]].shape
get_batch(data, lbl)[2].shape

(200,)

In [15]:
def get_siamese_net_and_encoder(input_shape, code_size = 0):
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    encoder = Sequential()
    encoder.add(Conv2D(64, (10, 10), input_shape=input_shape, activation='relu', kernel_regularizer='l2'))
    encoder.add(BatchNormalization())
    encoder.add(Activation('relu'))
    encoder.add(MaxPooling2D(pool_size=2))
    encoder.add(Dropout(0.25))
    
    encoder.add(Conv2D(128, (7, 7), kernel_regularizer='l2'))
    encoder.add(BatchNormalization())
    encoder.add(Activation('relu'))
    encoder.add(MaxPooling2D(pool_size=2))
    encoder.add(Dropout(0.25))
    
    encoder.add(Conv2D(128, (4, 4), kernel_regularizer='l2'))
    encoder.add(BatchNormalization())
    encoder.add(Activation('relu'))
    encoder.add(MaxPooling2D(pool_size=2))
    encoder.add(Dropout(0.25))
    #encoder.add(Conv2D(256, (4, 4), kernel_regularizer='l2'))
    
    encoder.add(Flatten())
    
    encoder.add(Dense(2048, activation='sigmoid', kernel_regularizer='l2'))
    
    left_emb = encoder(left_input)
    right_emb = encoder(right_input)
    
    L1_Layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    L1_Dist = L1_Layer([left_emb,right_emb])
    OP = Dense(1, activation='sigmoid', kernel_regularizer='l2')(L1_Dist)
    
    siamese_net = Model(inputs=[left_input, right_input], outputs=OP)
    
    return siamese_net, encoder


In [16]:
siamese_net, encoder = get_siamese_net_and_encoder((img_size, img_size, 1))

siamese_net.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

siamese_net.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 56, 56, 1)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 56, 56, 1)]  0           []                               
                                                                                                  
 sequential (Sequential)        (None, 2048)         1722176     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 2048)         0           ['sequential[0][0]',         

In [ ]:
def get_random_images(images):
    ind = np.arange(images.shape[0])
    ind = np.random.choice(ind, 1)[0]
    return images[ind]

def get_train_data(size, train_data, train_labels, img_size):
    targets = np.zeros((size,))
    targets[size // 2:] = 1
    pairs = [np.zeros((size, img_size, img_size, 1)) for _ in range(2)]
    labels = np.unique(train_labels)
    for i in range(size):
        class1 = np.random.choice(labels, 1)[0]
        class2 = class1
        if i < size // 2:
            while class2 == class1:
                class2 = np.random.choice(labels, 1)[0]
        pairs[0][i] = get_random_images(train_data[class1])
        pairs[1][i] = get_random_images(train_data[class2])
    return pairs, targets

def visualize_siamese(siamese_net, train_data, label1 = 1, label2=2):
    x = train_data[label1][:1]
    y = train_data[label2][1:2]
    d = siamese_net.predict([x, y], verbose=0)
    fig, axis = plt.subplots(1, 2, figsize=(10, 5))
    axis[0].axis("off")
    axis[1].axis("off")
    fig.suptitle(f"Output {d[0][0]}")
    axis[0].imshow(x[0], cmap='gray')
    axis[1].imshow(y[0], cmap='gray')
    plt.show()

In [18]:
train_data, train_labels = parse_omniglot_dataframe(df_train, img_size=img_size, reshape=False)
def train_one_shot():
    for i in range(1, num_mini_iterations + 1):
        print(f"Epoch {i}:")
        #x, y, _ = get_batch(train_data, train_labels, shots=shots, num_classes=num_classes, num_support=train_shots, img_size=img_size)
        x, y = get_train_data(2048, train_data, train_labels, img_size=img_size)
        loss = siamese_net.fit(x, y, batch_size=batch_size,)
        #if i % evaluateEvery == 0:
        #    print('Iteration', i, '- Loss:',loss[0],'- Acc:', round(loss[1], 2))
train_one_shot()

Epoch 1:
64/64 [==============================] - 5s 13ms/step - loss: 4.7342 - accuracy: 0.6309
Epoch 2:
64/64 [==============================] - 1s 12ms/step - loss: 1.3322 - accuracy: 0.6445
Epoch 3:
64/64 [==============================] - 1s 12ms/step - loss: 0.9485 - accuracy: 0.6821
Epoch 4:
64/64 [==============================] - 1s 12ms/step - loss: 0.8722 - accuracy: 0.6704
Epoch 5:
64/64 [==============================] - 1s 12ms/step - loss: 0.8662 - accuracy: 0.6602
Epoch 6:
64/64 [==============================] - 1s 12ms/step - loss: 0.7957 - accuracy: 0.6650
Epoch 7:
64/64 [==============================] - 1s 12ms/step - loss: 0.7796 - accuracy: 0.6738
Epoch 8:
64/64 [==============================] - 1s 12ms/step - loss: 0.7812 - accuracy: 0.6641
Epoch 9:
64/64 [==============================] - 1s 12ms/step - loss: 0.7466 - accuracy: 0.6670
Epoch 10:
64/64 [==============================] - 1s 12ms/step - loss: 0.7196 - accuracy: 0.6768
Epoch 11:
64/64 [============

64/64 [==============================] - 1s 12ms/step - loss: 0.7086 - accuracy: 0.7998
Epoch 85:
64/64 [==============================] - 1s 12ms/step - loss: 0.7084 - accuracy: 0.7915
Epoch 86:
64/64 [==============================] - 1s 12ms/step - loss: 0.7074 - accuracy: 0.8022
Epoch 87:
64/64 [==============================] - 1s 12ms/step - loss: 0.6758 - accuracy: 0.8096
Epoch 88:
64/64 [==============================] - 1s 12ms/step - loss: 0.7018 - accuracy: 0.7993
Epoch 89:
64/64 [==============================] - 1s 12ms/step - loss: 0.6866 - accuracy: 0.8115
Epoch 90:
64/64 [==============================] - 1s 12ms/step - loss: 0.6822 - accuracy: 0.8159
Epoch 91:
64/64 [==============================] - 1s 12ms/step - loss: 0.6983 - accuracy: 0.8076
Epoch 92:
64/64 [==============================] - 1s 12ms/step - loss: 0.6915 - accuracy: 0.8057
Epoch 93:
64/64 [==============================] - 1s 12ms/step - loss: 0.6635 - accuracy: 0.8174
Epoch 94:
64/64 [=============

64/64 [==============================] - 1s 12ms/step - loss: 0.6168 - accuracy: 0.8330
Epoch 168:
64/64 [==============================] - 1s 12ms/step - loss: 0.5990 - accuracy: 0.8486
Epoch 169:
64/64 [==============================] - 1s 12ms/step - loss: 0.6204 - accuracy: 0.8335
Epoch 170:
64/64 [==============================] - 1s 12ms/step - loss: 0.6072 - accuracy: 0.8350
Epoch 171:
64/64 [==============================] - 1s 12ms/step - loss: 0.6092 - accuracy: 0.8457
Epoch 172:
64/64 [==============================] - 1s 12ms/step - loss: 0.6165 - accuracy: 0.8403
Epoch 173:
64/64 [==============================] - 1s 12ms/step - loss: 0.6368 - accuracy: 0.8350
Epoch 174:
64/64 [==============================] - 1s 12ms/step - loss: 0.6165 - accuracy: 0.8457
Epoch 175:
64/64 [==============================] - 1s 12ms/step - loss: 0.6231 - accuracy: 0.8345
Epoch 176:
64/64 [==============================] - 1s 12ms/step - loss: 0.6358 - accuracy: 0.8276
Epoch 177:
64/64 [===

64/64 [==============================] - 1s 12ms/step - loss: 0.5726 - accuracy: 0.8579
Epoch 250:
64/64 [==============================] - 1s 12ms/step - loss: 0.5842 - accuracy: 0.8501
Epoch 251:
64/64 [==============================] - 1s 12ms/step - loss: 0.5864 - accuracy: 0.8496
Epoch 252:
64/64 [==============================] - 1s 12ms/step - loss: 0.6094 - accuracy: 0.8408
Epoch 253:
64/64 [==============================] - 1s 12ms/step - loss: 0.5863 - accuracy: 0.8462
Epoch 254:
64/64 [==============================] - 1s 12ms/step - loss: 0.5973 - accuracy: 0.8413
Epoch 255:
64/64 [==============================] - 1s 12ms/step - loss: 0.6130 - accuracy: 0.8311
Epoch 256:
64/64 [==============================] - 1s 12ms/step - loss: 0.5887 - accuracy: 0.8477
Epoch 257:
64/64 [==============================] - 1s 12ms/step - loss: 0.6057 - accuracy: 0.8442
Epoch 258:
64/64 [==============================] - 1s 12ms/step - loss: 0.5945 - accuracy: 0.8481
Epoch 259:
64/64 [===

64/64 [==============================] - 1s 12ms/step - loss: 0.5751 - accuracy: 0.8667
Epoch 332:
64/64 [==============================] - 1s 12ms/step - loss: 0.5744 - accuracy: 0.8584
Epoch 333:
64/64 [==============================] - 1s 12ms/step - loss: 0.5738 - accuracy: 0.8594
Epoch 334:
64/64 [==============================] - 1s 12ms/step - loss: 0.5901 - accuracy: 0.8525
Epoch 335:
64/64 [==============================] - 1s 12ms/step - loss: 0.6116 - accuracy: 0.8467
Epoch 336:
64/64 [==============================] - 1s 12ms/step - loss: 0.5831 - accuracy: 0.8535
Epoch 337:
64/64 [==============================] - 1s 12ms/step - loss: 0.5630 - accuracy: 0.8662
Epoch 338:
64/64 [==============================] - 1s 12ms/step - loss: 0.5861 - accuracy: 0.8550
Epoch 339:
64/64 [==============================] - 1s 12ms/step - loss: 0.5752 - accuracy: 0.8643
Epoch 340:
64/64 [==============================] - 1s 12ms/step - loss: 0.5838 - accuracy: 0.8589
Epoch 341:
64/64 [===

...

64/64 [==============================] - 1s 12ms/step - loss: 0.5724 - accuracy: 0.8516
Epoch 578:
64/64 [==============================] - 1s 12ms/step - loss: 0.5702 - accuracy: 0.8672
Epoch 579:
64/64 [==============================] - 1s 13ms/step - loss: 0.6022 - accuracy: 0.8540
Epoch 580:
64/64 [==============================] - 1s 12ms/step - loss: 0.5843 - accuracy: 0.8608
Epoch 581:
64/64 [==============================] - 1s 12ms/step - loss: 0.5869 - accuracy: 0.8584
Epoch 582:
64/64 [==============================] - 1s 12ms/step - loss: 0.5536 - accuracy: 0.8730
Epoch 583:
64/64 [==============================] - 1s 12ms/step - loss: 0.5887 - accuracy: 0.8613
Epoch 584:
64/64 [==============================] - 1s 12ms/step - loss: 0.6035 - accuracy: 0.8511
Epoch 585:
64/64 [==============================] - 1s 12ms/step - loss: 0.6002 - accuracy: 0.8540
Epoch 586:
64/64 [==============================] - 1s 12ms/step - loss: 0.5812 - accuracy: 0.8604
Epoch 587:
64/64 [===

64/64 [==============================] - 1s 12ms/step - loss: 0.5991 - accuracy: 0.8535
Epoch 660:
64/64 [==============================] - 1s 12ms/step - loss: 0.6025 - accuracy: 0.8608
Epoch 661:
64/64 [==============================] - 1s 12ms/step - loss: 0.5936 - accuracy: 0.8511
Epoch 662:
64/64 [==============================] - 1s 12ms/step - loss: 0.5832 - accuracy: 0.8599
Epoch 663:
64/64 [==============================] - 1s 12ms/step - loss: 0.5840 - accuracy: 0.8589
Epoch 664:
64/64 [==============================] - 1s 12ms/step - loss: 0.5842 - accuracy: 0.8525
Epoch 665:
64/64 [==============================] - 1s 12ms/step - loss: 0.5934 - accuracy: 0.8608
Epoch 666:
64/64 [==============================] - 1s 12ms/step - loss: 0.5570 - accuracy: 0.8716
Epoch 667:
64/64 [==============================] - 1s 12ms/step - loss: 0.6256 - accuracy: 0.8403
Epoch 668:
64/64 [==============================] - 1s 12ms/step - loss: 0.5866 - accuracy: 0.8633
Epoch 669:
64/64 [===

64/64 [==============================] - 1s 12ms/step - loss: 0.5620 - accuracy: 0.8706
Epoch 742:
64/64 [==============================] - 1s 12ms/step - loss: 0.5986 - accuracy: 0.8525
Epoch 743:
64/64 [==============================] - 1s 12ms/step - loss: 0.6119 - accuracy: 0.8540
Epoch 744:
64/64 [==============================] - 1s 12ms/step - loss: 0.5753 - accuracy: 0.8711
Epoch 745:
64/64 [==============================] - 1s 12ms/step - loss: 0.5766 - accuracy: 0.8652
Epoch 746:
64/64 [==============================] - 1s 12ms/step - loss: 0.5888 - accuracy: 0.8584
Epoch 747:
64/64 [==============================] - 1s 12ms/step - loss: 0.5683 - accuracy: 0.8589
Epoch 748:
64/64 [==============================] - 1s 12ms/step - loss: 0.5772 - accuracy: 0.8682
Epoch 749:
64/64 [==============================] - 1s 12ms/step - loss: 0.6047 - accuracy: 0.8457
Epoch 750:
64/64 [==============================] - 1s 12ms/step - loss: 0.5731 - accuracy: 0.8740
Epoch 751:
64/64 [===

64/64 [==============================] - 1s 12ms/step - loss: 0.5584 - accuracy: 0.8652
Epoch 824:
64/64 [==============================] - 1s 12ms/step - loss: 0.5941 - accuracy: 0.8491
Epoch 825:
64/64 [==============================] - 1s 12ms/step - loss: 0.5792 - accuracy: 0.8574
Epoch 826:
64/64 [==============================] - 1s 12ms/step - loss: 0.5967 - accuracy: 0.8579
Epoch 827:
64/64 [==============================] - 1s 12ms/step - loss: 0.5884 - accuracy: 0.8623
Epoch 828:
64/64 [==============================] - 1s 12ms/step - loss: 0.5893 - accuracy: 0.8608
Epoch 829:
64/64 [==============================] - 1s 12ms/step - loss: 0.6037 - accuracy: 0.8442
Epoch 830:
64/64 [==============================] - 1s 12ms/step - loss: 0.5847 - accuracy: 0.8677
Epoch 831:
64/64 [==============================] - 1s 12ms/step - loss: 0.5804 - accuracy: 0.8594
Epoch 832:
64/64 [==============================] - 1s 13ms/step - loss: 0.6042 - accuracy: 0.8491
Epoch 833:
64/64 [===

64/64 [==============================] - 1s 12ms/step - loss: 0.5725 - accuracy: 0.8687
Epoch 906:
64/64 [==============================] - 1s 12ms/step - loss: 0.5593 - accuracy: 0.8745
Epoch 907:
64/64 [==============================] - 1s 12ms/step - loss: 0.6149 - accuracy: 0.8467
Epoch 908:
64/64 [==============================] - 1s 12ms/step - loss: 0.5942 - accuracy: 0.8657
Epoch 909:
64/64 [==============================] - 1s 12ms/step - loss: 0.5828 - accuracy: 0.8628
Epoch 910:
64/64 [==============================] - 1s 12ms/step - loss: 0.5861 - accuracy: 0.8647
Epoch 911:
64/64 [==============================] - 1s 12ms/step - loss: 0.5900 - accuracy: 0.8662
Epoch 912:
64/64 [==============================] - 1s 12ms/step - loss: 0.5515 - accuracy: 0.8799
Epoch 913:
64/64 [==============================] - 1s 12ms/step - loss: 0.5725 - accuracy: 0.8628
Epoch 914:
64/64 [==============================] - 1s 12ms/step - loss: 0.5736 - accuracy: 0.8623
Epoch 915:
64/64 [===

In [28]:
print("oi")
visualize_siamese(siamese_net, train_images, train_labels, label1=63, label2=62)

In [20]:
# for alphabet in np.unique(df_test['alphabet']):
    #     ind_alphabet = np.where(df_test['alphabet'] == alphabet)[0]
    #     ind_alphabet = np.where(labels)
    #     labels = test_labels[ind_alphabet]
    #     images = test_images[ind_alphabet]
def test_siamese_random(test_data, test_labels, num_shots, num_classes):
    matches = 0
    total = 0
    def get_mode(l):
        d = {}
        mode = None
        count = 0
        for i in l:
            if i in d:
                d[i] += 1
            else:
                d[i] = 1
            if d[i] > count:
                mode = i
        return mode
    unique_labels = np.unique(test_labels)
    test_times = 500
    for it in range(test_times):
        classes = np.random.choice(unique_labels, num_classes, False)
        labels = []
        images = []
        for c in classes:
            images += [*test_data[c]]
            labels += [c for _ in range(len(test_data[c]))]
        images = np.array(images)
        labels = np.array(labels)
        os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=num_shots)

        x, y = [], []
        for i, c_i in enumerate(clas_img):
            x += [c_i.copy() for _ in range(len(os_img))]
            y = [*y, *os_img]
        x = np.array(x)
        y = np.array(y)
        #print(x.shape, y.shape)
        t = siamese_net.predict([x, y], verbose=0)
        for i in range(len(clas_img)):
            arr = t[i * len(os_img): (i + 1) * len(os_img)].reshape(-1)
            ind = np.argsort(arr)[-num_shots:]
            ind = get_mode(ind)
            pred = os_label[ind]
            #print(t.shape, ind, pred, clas_label[i])
            matches += np.sum(pred == clas_label[i])
            total += 1
        if it % 5 == 0:
            print(it,"Partial accuracy: ", matches/total)

    print("======= NL Autoencoder method: Finished =======")
    
test_data, test_labels = parse_omniglot_dataframe(df_test, img_size=img_size, reshape=False)
test_siamese_random(test_data, test_labels, num_shots=shots, num_classes=num_classes)

In [ ]:
def test_siamese_alphabets(test_images, test_labels, num_shots=shots):
    matches = 0
    total = 0
    def get_mode(l):
        d = {}
        mode = None
        count = 0
        for i in l:
            if i in d:
                d[i] += 1
            else:
                d[i] = 1
            if d[i] > count:
                mode = i
        return mode

    print("Vectorizing ...")
    for alphabet in np.unique(df_test['alphabet']):
        ind_alphabet = np.where(df_test['alphabet'] == alphabet)[0]
        labels = test_labels[ind_alphabet]
        images = test_images[ind_alphabet].reshape(-1, img_size, img_size, 1)
        os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=num_shots)

        x, y = [], []
        for i, c_i in enumerate(clas_img):
            x += [c_i.copy() for _ in range(len(os_img))]
            y = [*y, *os_img]
        x = np.array(x)
        y = np.array(y)
        print(x.shape, y.shape)
        t = siamese_net.predict([x, y], verbose=0)
        for i in range(len(clas_img)):
            arr = t[i * len(os_img): (i + 1) * len(os_img)].reshape(-1)
            ind = np.argsort(arr)[-num_shots:]
            ind = get_mode(ind)
            pred = os_label[ind]
            #print(t.shape, ind, pred, clas_label[i])
            matches += np.sum(pred == clas_label[i])
            total += 1
        print("Accuracy: ", matches/total)
    print("======= NL Autoencoder method: Finished =======")
    
(test_images, test_labels) = resize_images(df_test['image'], img_size) / 256, df_test['label'].to_numpy()
test_siamese_alphabets(test_images, test_labels)

In [ ]:
matches = total = 0
print(f"Trying the encoder with {num_shots} shots")
for alphabet in np.unique(df_test['alphabet']):
    ind_alphabet = np.where(df_test['alphabet'] == alphabet)[0]
    labels = test_labels[ind_alphabet]
    images = test_images[ind_alphabet]
    os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=num_shots)
    
    os_img = encoder.predict(os_img)
    clas_img = encoder.predict(clas_img)
    
    neigh = KNeighborsClassifier(n_neighbors = num_shots)
    neigh.fit(os_img, os_label)
    
    pred = neigh.predict(clas_img)
    matches += np.sum(pred == clas_label)
    total += len(clas_label)
print(matches/total)

In [ ]:
## total = matches = 0
os_img, os_label, clas_img, clas_label = separate_fewshot(test_images, test_labels, n=1)
os_img = encoder.predict(os_img)
clas_img = encoder.predict(clas_img)

#if verbose: print("Learning oneshot ...")
#nn = min(train, 5)
neigh = KNeighborsClassifier(n_neighbors = 1)
neigh.fit(os_img, os_label)

#if verbose: print("Predicting ...")
pred = neigh.predict(clas_img)

matches += np.sum(pred == clas_label)
total += len(clas_label)
print(matches / total)